<a id="introduction"></a>
## Introduction to Supervised Learning
#### By Paul Hendricks
-------

In this notebook, we will show to do GPU accelerated Supervised Learning in RAPIDS.  We will not cover SGD Regression at this time.

**Table of Contents**

* [Introduction to Supervised Learning](#introduction)
* [Linear Regression](#linear)
* [Ridge Regression](#ridge)
* [K Nearest Neighbors](#knn)
* [Setup](#setup)
* [Conclusion](#conclusion)

Before going any further, let's make sure we have access to `matplotlib`, a popular Python library for visualizing data.

In [ ]:
import os
import subprocess

try:
    import matplotlib
except ModuleNotFoundError:
    os.system('conda install -y matplotlib')



In [ ]:
try:
    import nvgpu
except ModuleNotFoundError:
    os.system('pip install -U nvgpu')
try:
    import GPUtil
except ModuleNotFoundError:
    os.system('pip install gputil')

<a id="setup"></a>
## Setup

This notebook was tested using the following Docker containers:

* `rapidsai/rapidsai-nightly:0.8-cuda10.0-devel-ubuntu18.04-gcc7-py3.7` from [DockerHub - rapidsai/rapidsai-nightly](https://hub.docker.com/r/rapidsai/rapidsai-nightly)

This notebook was run on the NVIDIA Tesla V100 GPU. Please be aware that your system may be different and you may need to modify the code or install packages to run the below examples. 

If you think you have found a bug or an error, please file an issue here: https://github.com/rapidsai/notebooks/issues

Before we begin, let's check out our hardware setup by running the `nvidia-smi` command.

In [ ]:
import nvgpu
nvgpu.gpu_info()

Next, let's see what CUDA version we have:

In [ ]:
os.system('nvcc --version')

Next, let's load some helper functions from `matplotlib` and configure the Jupyter Notebook for visualization.

In [ ]:
from matplotlib.colors import ListedColormap
import matplotlib.pyplot as plt


%matplotlib inline

<a id="linear"></a>
## Linear Regression

After our data has been preprocessed, we often want to build a model so as to understand the relationships between different variables in our data. Scikit-Learn is an incredibly powerful toolkit that allows data scientists to quickly build models from their data. Below we show a simple example of how to create a Linear Regression model.



In [ ]:
import numpy as np; print('NumPy Version:', np.__version__)


# create the relationship: y = 2.0 * x + 1.0
n_rows = 100000  # let's use 100 thousand data points
w = 2.0
x = np.random.normal(loc=0, scale=1, size=(n_rows,))
b = 1.0
y = w * x + b

# add a bit of noise
noise = np.random.normal(loc=0, scale=2, size=(n_rows,))
y_noisy = y + noise

We can now visualize our data using the `matplotlib` library.

In [ ]:
plt.scatter(x, y_noisy, label='empirical data points')
plt.plot(x, y, color='black', label='true relationship')
plt.legend()

We'll use the `LinearRegression` class from Scikit-Learn to instantiate a model and fit it to our data.

In [ ]:
import sklearn; print('Scikit-Learn Version:', sklearn.__version__)
from sklearn.linear_model import LinearRegression


# instantiate and fit model
linear_regression = LinearRegression()

In [ ]:
%%time

linear_regression.fit(np.expand_dims(x, 1), y)

In [ ]:
# create new data and perform inference
inputs = np.linspace(start=-5, stop=5, num=1000)
outputs = linear_regression.predict(np.expand_dims(inputs, 1))

Let's now visualize our empirical data points, the true relationship of the data, and the relationship estimated by the model. Looks pretty close!

In [ ]:
plt.scatter(x, y_noisy, label='empirical data points')
plt.plot(x, y, color='black', label='true relationship')
plt.plot(inputs, outputs, color='red', label='predicted relationship (cpu)')
plt.legend()

The mathematical operations underlying many machine learning algorithms are often matrix multiplications. These types of operations are highly parallelizable and can be greatly accelerated using a GPU. cuML makes it easy to build machine learning models in an accelerated fashion while still using an interface nearly identical to Scikit-Learn. The below shows how to accomplish the same Linear Regression model but on a GPU.

First, let's convert our data from a NumPy representation to a cuDF representation.

In [ ]:
import cudf; print('cuDF Version:', cudf.__version__)


# create a cuDF DataFrame
df = cudf.DataFrame({'x': x, 'y': y_noisy})
print(df.head())

Next, we'll load the GPU accelerated `LinearRegression` class from cuML, instantiate it, and fit it to our data.

In [ ]:
import cuml; print('cuML Version:', cuml.__version__)
from cuml.linear_model import LinearRegression as LinearRegression_GPU


# instantiate and fit model
linear_regression_gpu = LinearRegression_GPU()

In [ ]:
%%time

linear_regression_gpu.fit(df[['x']], df['y'])

We can use this model to predict values for new data points, a step often called "inference" or "scoring". All model fitting and predicting steps are GPU accelerated.

In [ ]:
# create new data and perform inference
new_data_df = cudf.DataFrame({'inputs': inputs})
outputs_gpu = linear_regression_gpu.predict(new_data_df[['inputs']])

Lastly, we can overlay our predicted relationship using our GPU accelerated Linear Regression model (green line) over our empirical data points (light blue circles), the true relationship (blue line), and the predicted relationship from a model built on the CPU (red line). We see that our GPU accelerated model's estimate of the true relationship (green line) is identical to the CPU based model's estimate of the true relationship (red line)!

In [ ]:
plt.scatter(x, y_noisy, label='empirical data points')
plt.plot(x, y, color='black', label='true relationship')
plt.plot(inputs, outputs, color='red', label='predicted relationship (cpu)')
plt.plot(inputs, outputs_gpu.to_array(), color='green', label='predicted relationship (gpu)')
plt.legend()

<a id="ridge"></a>
## Ridge Regression

Ridge extends LinearRegression by providing L2 regularization on the coefficients when predicting response y with a linear combination of the predictors in X. It can reduce the variance of the predictors, and improves the conditioning of the problem.

Below, we instantiate and fit a Ridge Regression model to our data.

In [ ]:
from cuml.linear_model import Ridge as Ridge_GPU


# instantiate and fit model
ridge_regression_gpu = Ridge_GPU()

In [ ]:
%%time

ridge_regression_gpu.fit(df[['x']], df['y'])

Similar to the `LinearRegression` model we fitted early, we can use the `predict` method to generate predictions for new data.

In [ ]:
outputs_gpu = ridge_regression_gpu.predict(new_data_df[['inputs']])

Lastly, we can visualize our `Ridge` model's estimated relationship and overlay it our the empirical data points.

In [ ]:
plt.scatter(x, y_noisy, label='empirical data points')
plt.plot(x, y, color='black', label='true relationship')
plt.plot(inputs, outputs, color='red', label='linear regression (cpu)')
plt.plot(inputs, outputs_gpu.to_array(), color='green', label='ridge regression (gpu)')
plt.legend()

<a id="knn"></a>
## K Nearest Neighbors

NearestNeighbors is a unsupervised algorithm where if one wants to find the “closest” datapoint(s) to new unseen data, one can calculate a suitable “distance” between each and every point, and return the top K datapoints which have the smallest distance to it.

We'll generate some fake data using the `make_moons` function from the `sklearn.datasets` module. This function generates data points from two equations, each describing a half circle with a unique center. Since each data point is generated by one of these two equations, the cluster each data point belongs to is clear. The ideal classification algorithm will identify two clusters and associate each data point with the equation that generated it. 

These data points are generated using a non-linear relationship - so using a linear regression approach won't adequately solve problem. Instead, we can use a distance-based algorithm K Nearest Neighbors to classify each data point.

First, let's generate out data.

In [ ]:
from sklearn.datasets import make_moons


X, y = make_moons(n_samples=int(1e3), noise=0.05, random_state=0)
print(X.shape)

Let's visualize our data:

In [ ]:
figure = plt.figure()
axis = figure.add_subplot(111)
axis.scatter(X[y == 0, 0], X[y == 0, 1], 
             edgecolor='black',
             c='lightblue', marker='o', s=40, label='cluster 1')

axis.scatter(X[y == 1, 0], X[y == 1, 1], 
             edgecolor='black',
             c='red', marker='s', s=40, label='cluster 2')
plt.legend()
plt.tight_layout()
plt.show()

Before we build a KNN classification model, we first have to convert our data to a cuDF representation.

In [ ]:
X_df = cudf.DataFrame()
for column in range(X.shape[1]):
    X_df['feature_' + str(column)] = np.ascontiguousarray(X[:, column])

y_df = cudf.Series(y)

Next, we'll instantiate and fit a nearest neighbors model using the `NearestNeighbors` class from cuML.

In [ ]:
from cuml.neighbors import NearestNeighbors


knn = NearestNeighbors()

In [ ]:
knn.fit(X_df)

Once our model has been built and fitted to the data, we can query the model for the `k` nearest neighbors to each data point. The query returns a matrix representating the distances of each data point to its nearest `k` neighbors as well as the indices of those neighbors.

In [ ]:
n_neighbors = 3

distances, indices = knn.kneighbors(X_df, k=n_neighbors)

We can iterate through each of our data points and do a majority vote to determine which class it belongs to.

In [ ]:
predictions = []

for i in range(indices.shape[0]):
    row = indices.iloc[i, :]
    vote = sum(y_df[j] for j in row) / n_neighbors
    predictions.append(1.0 * (vote > 0.5))

predictions = np.asarray(predictions).astype(np.float32)

Lastly, we can visualize the predictions from our K Nearest Neighbors classifier - we see that despite the non-linearity of the data, the algorithm does an excellent job of classifying the data.

In [ ]:
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(8, 3))


ax1.scatter(X[y == 0, 0], X[y == 0, 1],
            edgecolor='black',
            c='lightblue', marker='o', s=40, label='cluster 1')
ax1.scatter(X[y == 1, 0], X[y == 1, 1],
            edgecolor='black',
            c='red', marker='s', s=40, label='cluster 2')
ax1.set_title('empirical data points')


ax2.scatter(X[predictions == 0, 0], X[predictions == 0, 1], c='lightblue',
            edgecolor='black',
            marker='o', s=40, label='cluster 1')
ax2.scatter(X[predictions == 1, 0], X[predictions == 1, 1], c='red',
            edgecolor='black',
            marker='s', s=40, label='cluster 2')
ax2.set_title('KNN predicted classes')

plt.legend()
plt.tight_layout()
plt.show()

<a id="conclusion"></a>
## Conclusion

In this notebook, we showed to do GPU accelerated Supervised Learning in RAPIDS. 

To learn more about RAPIDS, be sure to check out: 

* [Open Source Website](http://rapids.ai)
* [GitHub](https://github.com/rapidsai/)
* [Press Release](https://nvidianews.nvidia.com/news/nvidia-introduces-rapids-open-source-gpu-acceleration-platform-for-large-scale-data-analytics-and-machine-learning)
* [NVIDIA Blog](https://blogs.nvidia.com/blog/2018/10/10/rapids-data-science-open-source-community/)
* [Developer Blog](https://devblogs.nvidia.com/gpu-accelerated-analytics-rapids/)
* [NVIDIA Data Science Webpage](https://www.nvidia.com/en-us/deep-learning-ai/solutions/data-science/)
